In [50]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup as BS
import re
import ast
import numpy as np
import html.parser

import os

import itertools

import json

#print(fastai.__version__)

In [51]:
def preprocess_sentence(string: str):
    string = html.unescape(string.strip())
    string = re.sub('<[^>]+>', '', string)
    return string

In [52]:
def get_quest_responses(link):
    html_ = urllib.request.urlopen(link).read()
    soup = BS(html_)
    for a in soup.find_all('script',{"src":True}):
        if a['src'].startswith('https'):
            URL = a['src']
    a = soup.find('script',{'src':False,'type':False})
    
    resps = urllib.request.urlopen(URL).read()
    soup = BS(resps,exclude_encodings=["\u00cdn","\u00e3"],)

    a = (soup.find_all()[0].text)#.replace('\\\\', '\\').encode().decode('utf-8')
    a = a.replace("\\\\",'\\')
    a = (bytes(a, 'utf-8').decode("unicode_escape"))
    a = a.strip().split("\n")
    dimensao = preprocess_sentence(a[2])
    empresa = preprocess_sentence(a[3])
    print("    "+empresa+","+dimensao)
    return a,empresa,dimensao
    

In [53]:
def get_respostas(str_list):
    for string in str_list:
        if "aplicar_respostas" in string:
            #regex = r"(?<=\[)([^]]+)(?=\])"
            #matches = re.findall(regex, string)
            return string
    return ""

def norm_answers(answers,quest_len):
    new_ans = {}
    for i in range(quest_len):
        try:
            new_ans[str(i)] = answers[str(i)]
        except:
            new_ans[str(i)] = []
    return new_ans



In [83]:
def checa_subperguntas(pergunta):
    return len(pergunta['sub'])

def get_pergunta(pergunta):
    perg = {}
    alternativas = []
    subpergs = []
    #print(pergunta)
    perg['enunciado'] = preprocess_sentence(pergunta['value']['data'][0]['value'])
    #print(pergunta)
    colsize = 0
    perg['colsize'] = 0
    perg['type'] = pergunta['value']['type']
    if pergunta['value']['type'] == 'TableUniqueChoice' or pergunta['value']['type'] == 'TableMultipleChoice':
        for item in pergunta['value']['data']:
            if item['name'] == 'alternative':
                for row in item['value']['row']:
                    colsize=0
                    for col in item['value']['col']:
                        colsize+=1
                        if "(" in col['value']:
                            alternativas.append(preprocess_sentence(row['value']+" - "+col['value']))
                        else:
                            alternativas.append(preprocess_sentence(row['value']+" - ("+col['value']+")"))
                    perg['colsize'] = colsize
    else:
        for alternativa in pergunta['value']['data'][3]['value']:
            if alternativa is not None:
                alternativas.append(preprocess_sentence(alternativa['value']))
    perg['alternativas'] = alternativas
    if checa_subperguntas(pergunta):
        for subpergunta in pergunta['sub']:
            subpergs += get_pergunta(subpergunta)
    #print(perg)
    return [perg] + subpergs
    

In [84]:
def flat_arr(ans, len_alt):
    ans = np.array(ans,dtype=object)
    dim_size = len_alt/ans.shape[0]
    new_ans = []
    count = 0
    for i,dim in enumerate(ans):
        i *= dim_size
        for n in dim:
            new_ans.append(i+int(n))
    return np.array(new_ans)



In [86]:
anos = ['2019','2020']

# URL = "https://respostas.iseb3.com.br/answer/aHR0cHM6Ly9zaXN0ZW1hLmlzZWIzLmNvbS5ici9xdWVzdGlvbmFyaW9zL2lu%0AZGV4LnBocD9zZWN1cmVVcmw9Q1hOUjQ0a2ZFc2E5LTlsYzVqRFZrQUlzWnA0%0AcXp4RC1KMGkwdFNkbXh3cVlGemEtcGFtdkV3bWo1NUljeDlxZXRHN0k5VmRy%0AYzRvR3JQQk9CZU5mN2JmZ1h6eVlIU0FCV00xaXB6Z0hLNFExUk16T0hvOUNq%0AX3FLekNxdnAzcS1maFZFQjF2N1hEbW1sbHRfeXRqVFFQZ19xRUlsdHVydDFS%0ANGs3b3l5YVpsNkFHeU9vRlJ3YUplcV82anpGUzRNVWo0VC1OTXZicmdsSVRC%0AS1BpS2VHajVycWg3SVhxa2tUcEdxUGtDWVB3akNZZVgxbDNnWUwzd3QwdUhO%0AbmlZUmQxT1FwZF93RGxUTzMzd3FQblBPZ19HLTdySklUQy1qQi1uOUdxYmhW%0AUVRFRmpSNFlUZ1JEYXRCWVpHcFJoQ0pCOUZzY1AzOU9ZVmljSVZQRlp2MklL%0AXzg1bHZLNTdDYnhlelNxREUxUUxZbVc1Sk81SWltWTFCNTZGaWVsb0Vl%0A"
# URL = "https://respostas.iseb3.com.br/cicles/2017"
# html = urllib.request.urlopen(URL).read()

for ano in anos:
    print(ano)
    try:
        os.mkdir(ano)
    except:
        pass
    site = "https://respostas.iseb3.com.br/cicles/"+ano
    site_html = urllib.request.urlopen(site).read()
    soup = BS(site_html)
    links_with_text = []
    for a in soup.find_all('a', href=True): 
        if a.text: 
            links_with_text.append(a['href'])
    for link in links_with_text:
        url2 = 'https://respostas.iseb3.com.br'+link
        page = urllib.request.urlopen(url2).read()
        soup = BS(page)
        
        links = []
        for a in soup.find_all('iframe'):
            links.append(a['src'])
        resps,empresa,dim = get_quest_responses(links[0])
        empresa = empresa.replace("\\/",".")
        url3 = urllib.request.urlopen(links[0]).read()
        soup = BS(url3)
        
        try:
            os.mkdir(ano+"/"+dim)
        except:
            pass
        resps = get_respostas(resps)
        #print(resps)
        #print("{"+(re.search('(?<=\{)([^}]+)(?=\})', resps).group(0))+"}")
        try:
            answers = ast.literal_eval("{"+(re.search('(?<=\{)([^}]+)(?=\})', resps).group(0))+"}")
        except:
            #print("bugou")
            lista = ast.literal_eval(re.findall(r'\(\[.*\]\)\;',resps)[0][1:-2])
            #print(lista)
            answers = {}
            for i,r in enumerate(lista):
                answers[str(i)] = r
        
        q = soup.find('script',{'src':False,'type':False})
        q = str(q)
        q = q[q.index('['):]
        q = q[:-11]
        q= json.loads(q)
        q = q[0:4]
        
        qst = []
        for criterio in q:
            #print(html.unescape(criterio['value']['data'][0]['value']))
            for indicador in criterio['sub']:
                #print("    "*2+preprocess_sentence(indicador['value']['data'][0]['value']))
                for pergunta in indicador['sub']:
                    #print(pergunta)
                    qst.append(get_pergunta(pergunta))
        q_flat = list(itertools.chain.from_iterable(qst))
        new_ans = norm_answers(answers,len(q_flat))
        assert(len(q_flat)==len(new_ans))
        #print(new_ans)
        for quest, ans in zip(q_flat,list(new_ans.values())):
            #print(quest,ans)
            if quest['type'] == "TableUniqueChoice":
                ans = np.array(ans).reshape((-1,1)).tolist()
                #print(quest,ans)    
            ans_vec = np.zeros(len(quest['alternativas']))
            if any(isinstance(el, list) for el in ans):
                ans = flat_arr(ans,len(quest['alternativas']))
            for resposta in ans:
                ans_vec[int(resposta)] = 1
            quest['answer'] = ans_vec.tolist()
        resp_df = pd.DataFrame(q_flat).drop(['colsize','type'],axis=1).apply(pd.Series.explode).rename({'answer':empresa},axis=1).groupby(['enunciado','alternativas']).sum().T
        resp_df.to_csv(ano+"/"+dim+"/"+empresa+".csv")


2019


/scratch/guilhermedrummond/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: invalid escape sequence '\/'
  


    AES Brasil Energia S.A.,Dimensão Geral
    AES Brasil Energia S.A.,Dimensão Natureza do Produto
    AES Brasil Energia S.A.,Dimensão Governança Corporativa
    AES Brasil Energia S.A.,Dimensão Econômico-Financeira
    AES Brasil Energia S.A.,Dimensão Ambiental A
    AES Brasil Energia S.A.,Dimensão Social
    AES Brasil Energia S.A.,Dimensão Mudanças Climáticas
    B2W - COMPANHIA DIGITAL,Dimensão Geral
    B2W - COMPANHIA DIGITAL,Dimensão Natureza do Produto
    B2W - COMPANHIA DIGITAL,Dimensão Governança Corporativa
    B2W - COMPANHIA DIGITAL,Dimensão Econômico-Financeira
    B2W - COMPANHIA DIGITAL,Dimensão Ambiental C
    B2W - COMPANHIA DIGITAL,Dimensão Social
    B2W - COMPANHIA DIGITAL,Dimensão Mudanças Climáticas
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Geral
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Natureza do Produto
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Governança Corporativa
    BANCO SANTANDER (BRASIL) S.A.,Dimensão Econômico-Financeira
    BANCO SANTANDER (B

In [40]:
# #links_with_text
# site = "https://respostas.iseb3.com.br/cicles/"+"2017"
# site_html = urllib.request.urlopen(site).read()
# soup = BS(site_html)
# links_with_text = []
# for a in soup.find_all('a', href=True): 
#     if a.text: 
#         links_with_text.append(a['href'])

In [43]:
# links_with_text[0]

'/answer/aHR0cHM6Ly9zaXN0ZW1hLmlzZWIzLmNvbS5ici9xdWVzdGlvbmFyaW9zL2lu%0AZGV4LnBocD9zZWN1cmVVcmw9Q1hOUjQ0a2ZFc2E5LTlsYzVqRFZrQUlzWnA0%0AcXp4RC1KMGkwdFNkbXh3b09ra2xwckRvUTFsZzNlQmNKQVIwMHMzMFpHcFhy%0AYUtVY2NDTGNmWlBKaTY0SDB4RWF6UElwblhYQ0htaG4zbk0xUk16T0hvOUNq%0AX3FLekNxdnAzcS1maFZFQjF2N1hEbW1sbHRfeXRqVFFQZ19xRUlsdHVydDFS%0ANGs3b3l5YVpsNkFHeU9vRlJ3YUplcV82anpGUzRNVWo0VC1OTXZicmdsSVRC%0AS1BpS2VHajVycWg3SVhxa2tUcEdxUGtDWVB3anFoSUk3dFZ5M1hOXzZuUmQ2%0AOEpLbzNCaGREZzNFX1czanBuRFpUWmxWUUJsVXhncU9SZDhidllwQ3doMEl3%0AdTFtRWR3WUdIaWFDd0F3d3oxeDNSUGVCOUZzY1AzOU9ZVmljSVZQRlp2MklL%0AXzg1bHZLNTdDYnhlelNxREUxUUxZbVc1Sk81SWltWTFCNTZGaWVsb0Vl%0A'

In [46]:
# url2 = 'https://respostas.iseb3.com.br'+links_with_text[0]

# html = urllib.request.urlopen(url2).read()

# soup = BS(html)

# links = []
# for a in soup.find_all('iframe'):
#     links.append(a['src'])
# links

['https://sistema.iseb3.com.br/questionarios/index.php?secureUrl=CXNR44kfEsa9-9lc5jDVkAIsZp4qzxD-J0i0tSdmxwoOkklprDoQ1lg3eBcJAR00s30ZGpXraKUccCLcfZPJi64H0xEazPIpnXXCHmhn3nM1RMzOHo9Cj_qKzCqvp3q-fhVEB1v7XDmmllt_ytjTQPg_qEIlturt1R4k7oyyaZl6AGyOoFRwaJeq_6jzFS4MUj4T-NMvbrglITBKPiKeGj5rqh7IXqkkTpGqPkCYPwjqhII7tVy3XN_6nRd68JKo3BhdDg3E_W3jpnDZTZlVQBlUxgqORd8bvYpCwh0Iwu1mEdwYGHiaCwAwwz1x3RPeB9FscP39OYVicIVPFZv2IK_85lvK57CbxezSqDE1QLYmW5JO5IimY1B56FieloEe']

In [144]:
# links

In [47]:
# #pega questionario (a) e respostas (URL)

# html2 = urllib.request.urlopen(links[0]).read()

# soup = BS(html2)
# for a in soup.find_all('script',{"src":True}):
#     if a['src'].startswith('https'):
#         URL = a['src']
# a = soup.find('script',{'src':False,'type':False})

In [145]:
# Processa respostas

# resps = urllib.request.urlopen(URL).read()
# print(URL)
# soup = BS(resps,exclude_encodings=["\u00cdn","\u00e3"],)

# import html.parser

# a = (soup.find_all()[0].text)#.replace('\\\\', '\\').encode().decode('utf-8')
# a = a.replace("\\\\",'\\')
# a = (bytes(a, 'utf-8').decode("unicode_escape"))
# a = a.strip().split("\n")

In [146]:
# precisa do a[2] e a[3]
# dimensao = preprocess_sentence(a[2])
# empresa = preprocess_sentence(a[3])
# print(dimensao,",",empresa)

In [136]:


# new_ans = norm_answers(answers,len(q_flat))
#new_ans

In [140]:
# resps = get_respostas(a)
# answers = ast.literal_eval("{"+(re.search('(?<=\{)([^}]+)(?=\})', resps).group(0))+"}")
# answers

In [195]:
# import html.parser

In [190]:
# soup = BS(html2)
# a = soup.find('script',{'src':False,'type':False})
# a = str(a)
# a = a[a.index('['):]
# a = a[:-11]

In [191]:
# import json

In [192]:
# a= json.loads(a)

In [205]:
# preprocess_sentence(a[0]['sub'][1]['sub'][0]['value']['data'][0]['value'])

In [102]:
#a[0]['sub'][0]['sub'][0]#['value']#['data']#[0]['value']

In [184]:
# a[3]['value']['data'][0]['value']

In [198]:
# a = a[0:4]

In [187]:
# a[0]['sub'][0]['sub'][0]['value']['data'][0]['value']#[0]['value']

In [188]:
# a[0]['sub'][0]['sub'][0]['sub'][3]['value']['data'][0]['value']

In [114]:
# def checa_subperguntas(pergunta):
#     return len(pergunta['sub'])

# def get_pergunta(pergunta):
#     perg = {}
#     alternativas = []
#     subpergs = []
#     perg['enunciado'] = preprocess_sentence(pergunta['value']['data'][0]['value'])
#     #print(pergunta)
#     colsize = 0
#     perg['colsize'] = 0
#     perg['type'] = pergunta['value']['type']
#     if pergunta['value']['type'] == 'TableUniqueChoice' or pergunta['value']['type'] == 'TableMultipleChoice':
#         for item in pergunta['value']['data']:
#             if item['name'] == 'alternative':
#                 for row in item['value']['row']:
#                     colsize=0
#                     for col in item['value']['col']:
#                         colsize+=1
#                         if "(" in col['value']:
#                             alternativas.append(preprocess_sentence(row['value']+" - "+col['value']))
#                         else:
#                             alternativas.append(preprocess_sentence(row['value']+" - ("+col['value']+")"))
#                     perg['colsize'] = colsize
#     else:
#         for alternativa in pergunta['value']['data'][3]['value']:
#             alternativas.append(preprocess_sentence(alternativa['value']))
#     perg['alternativas'] = alternativas
#     if checa_subperguntas(pergunta):
#         for subpergunta in pergunta['sub']:
#             subpergs += get_pergunta(subpergunta)
#     #print(perg)
#     return [perg] + subpergs
    

In [204]:
# qst = []
# for criterio in a:
#     #print(html.unescape(criterio['value']['data'][0]['value']))
#     for indicador in criterio['sub']:
#         print("    "+preprocess_sentence(indicador['value']['data'][0]['value']))
#         for pergunta in indicador['sub']:
#             qst.append(get_pergunta(pergunta))

In [116]:
# import itertools

In [201]:
# q_flat = list(itertools.chain.from_iterable(qst))

In [208]:
# def flat_arr(ans, len_alt):
#     ans = np.array(ans,dtype=object)
#     dim_size = len_alt/ans.shape[0]
#     new_ans = []
#     count = 0
#     for i,dim in enumerate(ans):
#         i *= dim_size
#         for n in dim:
#             new_ans.append(i+int(n))
#     return np.array(new_ans)

# tst = [[0,1,2],[0,1,2],[0,1,2]]
# flat_arr(tst,9)

In [207]:
# tst = np.array([2,2,2,2])
# tst.reshape((-1,1)).tolist()

In [221]:
# for quest, ans in zip(q_flat,list(new_ans.values())):
# #     print(quest,ans)
#     if quest['type'] == "TableUniqueChoice":
#         ans = np.array(ans).reshape((-1,1)).tolist()
#         #print(quest,ans)    
#     ans_vec = np.zeros(len(quest['alternativas']))
#     if any(isinstance(el, list) for el in ans):
#         ans = flat_arr(ans,len(quest['alternativas']))
#     for resposta in ans:
#         ans_vec[int(resposta)] = 1
#     quest['answer'] = ans_vec.tolist()
    

In [206]:
#(len(new_ans),len(q_flat))

In [155]:
#list(answers.values())

In [157]:
#q_flat

In [165]:
# #json.dumps(q_flat)
# with open('teste.json', 'w') as f:
#     json.dump(q_flat, f)
# test = pd.read_json('teste.json')

In [234]:
# df=pd.DataFrame(q_flat).drop(['colsize','type'],axis=1).apply(pd.Series.explode).rename({'answer':'empresa'},axis=1).groupby(['enunciado','alternativas'],sort=False).sum()

In [235]:
# df.T.to_csv("teste.csv")

In [236]:
# df = pd.read_csv("teste.csv",header=[0,1],index_col=0)
# df

enunciado    O compromisso com o Desenvolvimento Sustentável está formalmente inserido na cultura e na estratégia da companhia?  \
alternativas                                                                                                                Sim   
empresa                                                     1.0                                                                   

enunciado          \
alternativas  Não   
empresa       0.0   

enunciado    Se SIM para a PERGUNTA 1, este compromisso está formalmente expresso:  \
alternativas                        Na Visão, ou declaração de natureza semelhante   
empresa                                                     1.0                      

enunciado                                                     \
alternativas Na Missão, ou declaração de natureza semelhante   
empresa                                                  1.0   

enunciado                                                                                                  \
alternativas No Código de Conduta que explicite valores e princípios que norteiam a operação da companhia   
empresa                                                     1.0                                             

enunciado                                  \
alternativas Nas políticas da organização   
empresa                               1.0   

enunciado                                                                                                \
alternativas Nos objetivos e metas de desempenho socioambiental assumidos em documento público e formal   
empresa                                                     1.0                                           

enunciado                            \
alternativas Nenhuma das anteriores   
empresa                         0.0   

enunciado    Se SIM para a PERGUNTA 1, assinale os temas expressamente contemplados neste compromisso:  \
alternativas                                                                 Respeito ao meio ambiente   
empresa                                                     1.0                                          

enunciado                                ...  \
alternativas Práticas leais de operação  ...   
empresa                             1.0  ...   

enunciado                                 Se SIM para a PERGUNTA 29, o relatório foi:  \
alternativas Integralmente disponibilizado em pelo menos um idioma, além do português   
empresa                                                     1.0                         

enunciado                                                                                                                           \
alternativas Divulgado em versão adaptada especificamente a partes interessadas que têm necessidades específicas de acessibilidade   
empresa                                                     0.0                                                                      

enunciado                                                                                                                                                                                          \
alternativas Divulgado em versões adicionais, que visem à sua compreensão e estimulem sua leitura pelas diferentes partes interessadas, considerando suas peculiaridades e interesses específicos   
empresa                                                     0.0                                                                                                                                     

enunciado                            \
alternativas Nenhuma das anteriores   
empresa                         0.0   

enunciado    Se SIM para a PERGUNTA 29, existe envolvimento direto dos administradores da companhia na definição do Relatório de Sustentabilidade?  \
alternativas                                                                                                                                   Sim   
empresa                                  

In [217]:
# os.mkdir("a")

FileExistsError: [Errno 17] File exists: 'a'

In [11]:
# t ="""$.fn.aplicar_respostas(["0",["0","1"],["0","1","2","3","4","5","6","7","8"],["0","1","2","3","4"],["0","1","2","3","4","5","6","7","8"],["2","3"],"0",["0","1","2","3","4"],"0","0",["0","1"],"0","3","0","0","0","2",["1","2","3","4"],"0",["0","1","2","3","4","5","6","7","8"],["0","1","2","3","4","5"],"3","0",["0","1"],"0","0","1","0",["0","1","2","3","4","5"],[["0","1"],["0","1"],["0","1"]],["0","1","2","3","4"],["0","2"],"0","1","2","1",["1","2"],"0",["0","1","2"],["0","1","2","3","4","5","6","7","8","9","11"],["0","1","3"],["0","1","2","3","4","5","6","7","8"],["0","1","2","3","4"],"0",["0","1","2","3","4","5"]]); $("#questionnaire li input").each(function(){$(this).hide(); if($(this).prop("checked")){$(this).after($("").addClass("icon-ok green")); }else{$(this).after($("").addClass("icon-")); }}); $("h2.nome").css({"display":"inline"}); $("strong").each(function(){ $(this).before(" "+$(this).text()); $(this).remove(); }); """

In [32]:
# lista = ast.literal_eval(re.findall(r'\(\[.*\]\)\;',t)[0][1:-2])

In [34]:
# dc

{'0': '0',
 '1': ['0', '1'],
 '2': ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 '3': ['0', '1', '2', '3', '4'],
 '4': ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 '5': ['2', '3'],
 '6': '0',
 '7': ['0', '1', '2', '3', '4'],
 '8': '0',
 '9': '0',
 '10': ['0', '1'],
 '11': '0',
 '12': '3',
 '13': '0',
 '14': '0',
 '15': '0',
 '16': '2',
 '17': ['1', '2', '3', '4'],
 '18': '0',
 '19': ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 '20': ['0', '1', '2', '3', '4', '5'],
 '21': '3',
 '22': '0',
 '23': ['0', '1'],
 '24': '0',
 '25': '0',
 '26': '1',
 '27': '0',
 '28': ['0', '1', '2', '3', '4', '5'],
 '29': [['0', '1'], ['0', '1'], ['0', '1']],
 '30': ['0', '1', '2', '3', '4'],
 '31': ['0', '2'],
 '32': '0',
 '33': '1',
 '34': '2',
 '35': '1',
 '36': ['1', '2'],
 '37': '0',
 '38': ['0', '1', '2'],
 '39': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '11'],
 '40': ['0', '1', '3'],
 '41': ['0', '1', '2', '3', '4', '5', '6', '7', '8'],
 '42': ['0', '1', '2', '3', '4'],
 '43': '0',
 '4

In [98]:
url2 = 'https://respostas.iseb3.com.br/cicles/2015'

ht = urllib.request.urlopen(url2).read()

soup = BS(ht)

soup.find_all("td", {"class": ["tit_holding","tit_controlada"]})

[<td class="tit_holding">AES TIETE S.A.</td>,
 <td class="tit_holding">B2W - COMPANHIA DIGITAL</td>,
 <td class="tit_holding">BANCO SANTANDER (BRASIL) S.A.</td>,
 <td class="tit_holding">BCO BRADESCO S.A.</td>,
 <td class="tit_controlada">Bradesco Seguros S.A.</td>,
 <td class="tit_holding">BCO BRASIL S.A.</td>,
 <td class="tit_holding">BRASKEM S.A.</td>,
 <td class="tit_holding">BRF S.A.</td>,
 <td class="tit_holding">CCR S.A.</td>,
 <td class="tit_controlada">Concessionária da Rodovia Presidente Dutra S.A.</td>,
 <td class="tit_controlada">Concessionária de Rodovias do Oeste de São Paulo - ViaOeste S.A.</td>,
 <td class="tit_controlada">Concessionária do Sistema Anhanguera - Bandeirantes S.A.</td>,
 <td class="tit_controlada">Rodonorte - Concessionária de Rodovias Integradas S.A.</td>,
 <td class="tit_controlada">Rodovias Integradas do Oeste S.A.</td>,
 <td class="tit_holding">CENTRAIS ELET BRAS S.A. - ELETROBRAS</td>,
 <td class="tit_controlada">Amazonas Distribuidora de Energia S.A